## Index Tables Data Export Codes

- 1. tbl_unit
- 2. tbl_log_parameters

In [1]:
# import required libraries
import numpy as np
import pandas as pd
import os
import sys
import mariadb
import warnings
warnings.filterwarnings("ignore")

In [14]:
# set file name
file_name = '..\..\..\dataExport\LOGdata\log_2022_4_20.csv'

In [15]:
# load data into a dataframe
log_rawDF = pd.read_csv(file_name, sep='*')

In [16]:
# extract uniques units and parameters from the log data 
# to enterthem into the index tables
def log_prm_unit_values(log_rawDF):
    log_rawDF = log_rawDF.dropna(subset=['Parameter'])
    log_parameter = log_rawDF['Parameter'].unique()
    tmp_1 = log_rawDF.query('Type != (1)')
    tmp_1 = tmp_1.query('Unit == ("FPC11", "FPC12", "FPC13", "FPC14", "FPC21", "FPC22", "FPC23", "FPC24", "FFU", "FPP", "Product", "scheduler")')
    log_unit = tmp_1['Unit'].unique()
    return ([log_unit, log_parameter])

tbl_values = log_prm_unit_values(log_rawDF)

print(tbl_values[0])
print(tbl_values[1])

['FPC14' 'FFU' 'FPC22' 'FPC21' 'FPC23' 'FPC13' 'FPC24' 'FPC11' 'Product'
 'FPC12' 'FPP']
['Arduino' 'program' 'T61' 'water' 'T10' 'Reactor' 'T71' 'pneumatic'
 'autoSyncMonitor' 'T42A' 'harvest' 'calculations' 'P1L' 'feeding' 'T64'
 'T42' 'T53' 'T51' 'T73' 'T74' 'T65' 'T63' 'T72' 'T52' 'T54' 'PT1_V4'
 'T55' 'T43' 'LEDlighting' 'RM29' 'RM6' 'V74' 'RM12' 'RM13' 'RM14' 'RM15'
 'RM16' 'RM17' 'RM19' 'RM2' 'RM21' 'RM22' 'RM23' 'T41A' 'T43A' 'RM35'
 'RM4' 'RM5' 'RM7' 'P42' 'RM1' 'RM30' 'Arduino_QG' 'Arduino_pH' 'V3' 'P1R'
 'P2R' 'CO2_V2' 'R1' 'PTdiff_K2' 'L1W' 'Arduino_TT1' 'Arduino_LT1'
 'Arduino1_LT52' 'Arduino2_LT72' 'Arduino1_LT41' 'Arduino2_LT61'
 'AutoLink' 'P43' 'V42' 'P10' 'V11' 'V8' 'LT1' 'RM3' 'RM11' 'RM8' 'M1'
 'P2L' 'V61' 'P11' 'Recipes' 'Products' 'L1' 'V6' 'V1' 'V72' 'chiller'
 'T62' 'V71' 'V62' 'P12' 'V43' 'L1FR' 'L1R' 'L1B' 'LT10' 'M10' 'V63'
 'RM18' 'Arduino1' 'V52' 'V64' 'PT10' 'TT0' 'waste' 'V51' 'RM20' 'RM33'
 'AutoTemplateSync' 'Arduino2_LT73' 'Arduino_CO2' 'Arduino2_LT63'

In [24]:
def db_connection():
    try: con = mariadb.connect( 
        user="root", 
        password="password", 
        host="localhost", 
        port=3306, 
        database="data_dashboard" 
        )

    except mariadb.Error as ex: 
        print(f"An error occurred while connecting to MariaDB: {ex}") 
        sys.exit(1) 

    return con.cursor(),con

#cur, con = db_connection()

In [25]:
# code to close the data base connection
def db_close(con):
    con.close()

In [26]:
# just a function for test the db connection is working or not
def test_query():
    cur, con = db_connection()
    cur.execute("SELECT unit_id, unit_name FROM tbl_unit")
    for (unit_id, unit_name) in cur:
        print("ID:", {unit_id}, "unit_name:", {unit_name})

    db_close(con)

test_query()

ID: {2} unit_name: {'FFU'}
ID: {8} unit_name: {'FPC11'}
ID: {10} unit_name: {'FPC12'}
ID: {6} unit_name: {'FPC13'}
ID: {1} unit_name: {'FPC14'}
ID: {4} unit_name: {'FPC21'}
ID: {3} unit_name: {'FPC22'}
ID: {5} unit_name: {'FPC23'}
ID: {7} unit_name: {'FPC24'}
ID: {11} unit_name: {'FPP'}
ID: {9} unit_name: {'Product'}


In [29]:
# Insert unique units into index table tbl_unit
def insrt_tbl_unit(log_unit):
    try:
        cur, con = db_connection()
        query = 'INSERT IGNORE INTO tbl_unit (unit_id, unit_name) VALUES (%s, %s)'
        tuples = pd.Series(log_unit)
        tuples = [(index+1, value) for index, value in tuples.items()]
        cur.executemany(query, tuples)
        con.commit()
        db_close(con)
    except Exception as e: print(e)

insrt_tbl_unit(tbl_values[0])

In [30]:
# Insert unique parameters into index table : tbl_log_parameters
def insrt_tbl_parameter(log_parameter):
    try:
        cur, con = db_connection()
        query = 'INSERT IGNORE INTO tbl_log_parameters (prm_id, prm_name) VALUES (%s, %s)'
        tuples = pd.Series(log_parameter)
        tuples = [(index+1, value) for index, value in tuples.items()]
        cur.executemany(query, tuples)
        con.commit()
        db_close(con)
    except Exception as e: print(e)

insrt_tbl_parameter(tbl_values[1])

## PBR lookup data preprocessing & DB export.

- Table       : tbl_pbr_lookup
- File name   : tbl_pbr_lookup.xlsx

Operations : 
1. Load the data 
2. populate unit and species column with DB tables primary key
3. Export data into the pbr_lookpup table

In [2]:
import sqlalchemy

In [3]:
# set the file path
file_name = file_name = '..\..\..\dataExport\\tbl_pbr_lookup.xlsx'

In [4]:
# load the excel data sheet and display first 5 rows
pbr_lookup = pd.read_excel(file_name,sheet_name='Sheet1')
pbr_lookup

,instance,unit,species,start_date,end_date,remark
0,1.0,FPC13,1.0,1.629468e+09,1.633939e+09,NaN
1,2.0,FPC13,1.0,1.634652e+09,1.644489e+09,NaN
2,3.0,FPC13,11.0,1.625227e+09,1.628158e+09,NaN
3,4.0,FPC14,2.0,1.614764e+09,1.618393e+09,NaN
4,5.0,FPC14,7.0,1.625486e+09,1.639564e+09,NaN
5,6.0,FPC14,11.0,1.618562e+09,1.625229e+09,NaN
6,7.0,FPC21,2.0,1.634915e+09,1.646645e+09,NaN
7,8.0,FPC21,9.0,1.623051e+09,1.626772e+09,NaN
8,9.0,FPC22,3.0,1.620216e+09,1.634632e+09,NaN
9,10.0,FPC23,7.0,1.616427e+09,1.626691e+09,NaN


In [5]:
def dbConn_sqlAlch():
    try:
        database_username = 'root'
        database_password = 'password'
        database_ip       = '127.0.0.1:3306'
        database_name     = 'data_dashboard'
        database_connection = sqlalchemy.create_engine('mariadb+mariadbconnector://{0}:{1}@{2}/{3}'.
                                                    format(database_username, database_password, 
                                                            database_ip, database_name))
        return database_connection
    except Exception as e: print(e)


database_connection = dbConn_sqlAlch()
    

In [6]:
# check DB connection is fine and fetch unique uint ID
tbl_unit = pd.read_sql('SELECT unit_id, unit_name FROM tbl_unit', database_connection)
tbl_unit.head(3)



,unit_id,unit_name
0,2,FFU
1,8,FPC11
2,10,FPC12


In [7]:
# replace unit names with primary key value
pbr_lookup['unit'] = pbr_lookup['unit'].map(tbl_unit.set_index('unit_name')['unit_id'])
pbr_lookup


,instance,unit,species,start_date,end_date,remark
0,1.0,6,1.0,1.629468e+09,1.633939e+09,NaN
1,2.0,6,1.0,1.634652e+09,1.644489e+09,NaN
2,3.0,6,11.0,1.625227e+09,1.628158e+09,NaN
3,4.0,1,2.0,1.614764e+09,1.618393e+09,NaN
4,5.0,1,7.0,1.625486e+09,1.639564e+09,NaN
5,6.0,1,11.0,1.618562e+09,1.625229e+09,NaN
6,7.0,4,2.0,1.634915e+09,1.646645e+09,NaN
7,8.0,4,9.0,1.623051e+09,1.626772e+09,NaN
8,9.0,3,3.0,1.620216e+09,1.634632e+09,NaN
9,10.0,5,7.0,1.616427e+09,1.626691e+09,NaN


In [9]:
try:
    pbr_lookup.to_sql(con=database_connection, name='tbl_pbr_lookup', if_exists='append', index=False)
except Exception as e: print(e)

(mariadb.IntegrityError) Duplicate entry '1' for key 'PRIMARY'
[SQL: INSERT INTO tbl_pbr_lookup (instance, unit, species, start_date, end_date, remark) VALUES (?, ?, ?, ?, ?, ?)]
[parameters: ((1.0, 6, 1.0, 1629468000.0, 1633939195.0, None), (2.0, 6, 1.0, 1634652000.0, 1644488995.0, None), (3.0, 6, 11.0, 1625227200.0, 1628157595.0, None), (4.0, 1, 2.0, 1614763800.0, 1618392595.0, None), (5.0, 1, 7.0, 1625486400.0, 1639564200.0, None), (6.0, 1, 11.0, 1618561800.0, 1625228995.0, None), (7.0, 4, 2.0, 1634914800.0, 1646645395.0, None), (8.0, 4, 9.0, 1623051000.0, 1626771600.0, None)  ... displaying 10 of 12 total bound parameter sets ...  (11.0, 7, 8.0, 1620129600.0, 1633964395.0, None), (12.0, 7, 8.0, 1635246000.0, 1641567595.0, None))]
(Background on this error at: https://sqlalche.me/e/14/gkpj)
